In [ ]:
import numpy as np

# Day 7: Symmetric and Banded Coefficient Matrices

Applications often involve coefficient matrices that are *sparse*ly populated. That means most of their entries are $0$'s. If all of the non-zero entries are clustered near the main diagonal, then the matrix is said to be *banded*. For example, see the matrix below, which is a tri-diagonal, banded matrix.

$$\left[\begin{array}{ccccccc}X & X & 0 & 0 & \cdots & 0 & 0 & 0\\
X & X & X & 0 & \cdots & 0 & 0 & 0\\
0 & X & X & X & \cdots & 0 & 0 & 0\\[10pt]
\vdots & \vdots & \vdots & \ddots & \ddots & \ddots & \vdots & \vdots\\[10pt]
0 & 0 & 0 & 0 &\cdots & X & X & X\\
0 & 0 & 0 & 0 & \cdots & 0 & X & X
\end{array}\right]$$

Those $X$ entries could be zero or non-zero, but all entries off of those three diagonals are $0$'s in a tri-diagonal matrix. We say that such a matrix has a *bandwidth* of $3$ since there are at most three non-zero elements in each row.

A nice property of these banded matrices is that, if they are *LU-decomposed*, then both $L$ and $U$ retain the banded structure. This structure can be exploited to save on both *memory requirements* and *run time*.

## An Example

To motivate our discovery and implementation of this method, let's try solving a system with a tridiagonal coefficient matrix.

**Example:** Solve the system
$$A = \left[\begin{array}{ccccc} 1 & 2 & 0 & 0 & 0\\
2 & -1 & 8 & 0 & 0\\
0 & 3 & -1 & -1 & 0\\
0 & 0 & 3 & 2 & -1\\
0 & 0 & 0 & 5 & -4\end{array}\right]\left[\begin{array}{c} x_1\\ x_2\\ x_3\\ x_4\\ x_5\end{array}\right] = \left[\begin{array}{c}13\\ 30\\ 7\\ 12\\ 6\end{array}\right]$$

> *Solution.*

### Doolittle's Method for Tridiagonal Coefficient Matrices

Consider the system $A\vec{x} = \vec{b}$, where $A$ is a tridiagonal matrix of the following form:

$$A = \left[\begin{array}{cccccc} d_1 & e_1 & 0 & 0 & \cdots & 0\\
c_1 & d_2 & e_2 & 0 & \cdots & 0\\
0 & c_2 & d_3 & e_3 & \cdots & 0\\
0 & 0 & c_3 & d_4 & \cdots & 0\\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots\\
0 & 0 & \cdots & 0 & c_{n-1} & d_n
\end{array}\right]$$

In this case, the majority of the entries will be $0$ (as long as the matrix is large enough). It becomes more efficient to store the three diagonal vectors only, since we know that all other entries are $0$'s. That is, instead of storing $A$ explicitly, we'll store:

$$\vec{c} = \left[\begin{array}{c} c_1\\ c_2\\ \vdots\\ c_{n-1}\end{array}\right]~~~\vec{d} = \left[\begin{array}{c} d_1\\ d_2\\ \vdots\\ d_n\end{array}\right]~~~\vec{e} = \left[\begin{array}{c} e_1\\ e_2\\ \vdots\\ e_{n-1}\end{array}\right]$$

This is a significant savings, since a $100\times 100$ tridiagonal matrix has $10,000$ entries, but we can get away with storing only $99 + 100 + 99 = 298$ entries using this vector storage trick. This is a *compression* of around $33:1$.

We can still apply *LU decomposition* to the coefficient "matrix" (really, the coefficient vectors) as long as we carefully track which vectors we are working with. As a reminder, the *LU-decomposition* begins with the usual *Gaussian Elimination* procedure. That is, to reduce $R_k$ (row $k$) we eliminate $c_{k-1}$ using:

$$R_k \leftarrow R_k - \left(c_{k-1}/d_{k-1}\right)R_{k-1}$$

Notice that in doing this, only $\vec{c}$ and $\vec{d}$ are changed. The changes are:

$$\begin{array}{lcl} d_k &\leftarrow &d_k - \left(c_{k-1}/d_{k-1}\right)e_{k-1}\\
c_{k-1} &\leftarrow & 0
\end{array}$$

At the end of the process, the vector $\vec{c}$ would be zeroed out. It is a waste of space to store that zero-vector, but $\vec{c}$ is a perfect place to store our $\lambda$ scalar multipliers for our pivot rows. For this reason, we'll update:

$$\begin{array}{lcl} d_k &\leftarrow &d_k - \left(c_{k-1}/d_{k-1}\right)e_{k-1}\\
c_{k-1} &\leftarrow & c_{k-1}/d_{k-1}
\end{array}$$

Thus, we have the following decomposition algorithm:

```
#tridiagonal LU decomp
for k in range(___, ___):
  lam = ___[___]/___[___]
  ___[___] = ___[___] - ___*___[___]
  ___[___] = ___
```

Now we'll look to the *solution phase*. As a reminder, we're solving $A\vec{x} = \vec{b}$ by solving $LU\vec{x} = \vec{b}$. The strategy is to use forward-substitution to solve $L\vec{y} = \vec{b}$ and then backward-substitution to solve $U\vec{x} = \vec{y}$ as before, but we need to deal with the fact that our factorized matrix elements are split across the three vectors $\vec{c}$, $\vec{d}$, and $\vec{e}$.

We can solve $L\vec{y} = \vec{b}$ as follows (remember that $\vec{c}$ contains our $\lambda$ multipliers):

```
#forward substitution
y[0] = b[0]
for k in range(___, ___):
  #Can use b to store values of y, since the
  #original entris of b are no longer needed
  b[k] = b[k] - ___[___]*___[___]
```

Now we solve $U\vec{x} = \vec{y}$ using backward-substitution.

```
#backward substitution
x[n-1] = y[n-1]/d[n-1]
for k in range(n-2, -1, -1):
  #Similarly, can use b to store values of x
  b[k] = (b[k] - ___[___]*___[___])/___[___]
```

We can put this all together into the `DoolittleLUdecomp3solver()` routine below.

In [ ]:
def DoolittleLUdecomp3(c, d, e):
  n = ___
  for k in range(___, ___):
    lam = ___[___]/___[___]
    ___[___] = ___[___] - ___*___[___]
    ___[___] = ___

  return c, d, e

def Doolittle3solver(lam, d, e, b):
  n = ___
  #Forward Substitution
  for k in range(___, ___):
    b[k] = b[k] - ___[___]*___[___]

  b[n-1] = b[n-1]/d[n-1]
  for k in range(n-2, -1, -1):
    b[k] = (b[k] - ___[___]*___[___])/___[___]

  return b

def DoolittleLUdecomp3solver(c, d, e, b):
  lam, d, e = ___
  b = ___

  return ___

**Example:** Verify that this solver works on our example from earlier! As a reminder, our system corresponded to the augmented coefficient matrix

$$A = \left[\begin{array}{ccccc} 1 & 2 & 0 & 0 & 0\\
2 & -1 & 8 & 0 & 0\\
0 & 3 & -1 & -1 & 0\\
0 & 0 & 3 & 2 & -1\\
0 & 0 & 0 & 5 & -4\end{array}\right]\left[\begin{array}{c} x_1\\ x_2\\ x_3\\ x_4\\ x_5\end{array}\right] = \left[\begin{array}{c}13\\ 30\\ 7\\ 12\\ 6\end{array}\right]$$

## Aside: Symmetric Coefficient Matrices and Increased Efficiency

Because of how often applications result in symmetric, banded coefficient matrices (where $\vec{c} = \vec{e}$), mathematicians and engineers spend time developing specialized approaches that work more quickly and efficiently than general approaches. For example, it can be shown that, if a matrix $A$ is symmetric, then its *LU-decomposition* can be written as

$$A = LU = LDL^T$$

where $D$ is a *diagonal matrix*. This means that we can still use Doolittle's decomposition with $U = DL^T$. That is,

\begin{align*} U &= DL^T\\
&= \left[\begin{array}{ccccc} D_1 & 0 & 0 & \cdots & 0\\
0 & D_2 & 0 & \cdots & 0\\
0 & 0 & D_3 & \cdots & 0\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
0 & 0 & 0 & \cdots & D_n
\end{array}\right]\left[\begin{array}{ccccc} 1 & L_{21} & L_{31} & \cdots & L_{n1}\\
0 & 1 & L_{32} & \cdots & L_{n2}\\
0 & 0 & 1 & \cdots & L_{n3}\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
0 & 0 & 0 & \cdots & 1
\end{array}\right]\\
&= \left[\begin{array}{ccccc} D_1 & D_1L_{21} & D_1L_{31} & \cdots & D_1L_{n1}\\
0 & D_2 & D_2L_{32} & \cdots & D_2L_{n2}\\
0 & 0 & D_3 & \cdots & D_3L_{n3}\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
0 & 0 & 0 & \cdots & D_n
\end{array}\right]
\end{align*}

Again, as a space-saving efficiency, we can choose to only store $U$ since both $L$ and $D$ are recoverable from this matrix! Again, the *Gaussian Elimination* procedure which results in an upper triangular matrix is sufficient to decompose a symmetric matrix.

There is an alternative, however, which is even more efficient due to speed-ups at the solution phase (since the $L_{ij}$ are not masked). We can instead store

$$U^* = \left[\begin{array}{ccccc} D_1 & L_{21} & L_{31} & \cdots & L_{n1}\\
0 & D_2 & L_{32} & \cdots & L_{n2}\\
0 & 0 & D_3 & \cdots & L_{n3}\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
0 & 0 & 0 & \cdots & D_n
\end{array}\right]$$

and notice that $U_{ij} = D_iL_{ji} = U^*_{ii}*U^*_{ij}$.

We won't build an implementation to specifically handle symmetric matrices here because our `LUdecomp3solver()` routine will handle all of the instances for our course. I include this aside however to point out that there are researchers and scientist who spend their careers investigating, developing, and implementing ways to improve run-time-complexity and space-complexity of existing algorithms or developing brand new algorithms.

***

## Summary

In this notebook, we considered additional efficiencies that can be gained by exploiting properties of the coefficient matrix. In particular, we looked at the special case of banded matrices. We could construct specialty solvers for other classes of coefficient matrix by analyzing the process involved in using the *Gaussian Elimination* approach to solving the corresponding system. Taking an algorithmic approach is much slower for a single problem, but pays off because we end up with a constructed routine that can "quickly" solve any problem fitting the assumptions of the algorithm we've built.